In [100]:
import numpy as np
import scipy.io.wavfile
from scikits.talkbox.features import mfcc
import glob
import os

path_to_train_audio_file = "/home/anjani/AMP/assignment_3/lang_rec_data_with_division/train"

all_lang_path = [x[0] for x in os.walk(path_to_train_audio_file)][1:]

all_language_features = {}

for path in all_lang_path:
    all_audio_files = glob.glob(path+"/*.wav")

    lang_audio_features = []
    
    for i in all_audio_files:
        sample_rate, X = scipy.io.wavfile.read(i)
        ceps, mspec, spec = mfcc(X)
        num_mspec = len(mspec)
        if path.rsplit('/',1)[1] == 'odia' :
            lang_audio_features.extend(mspec[int(3 * num_mspec / 10):int(num_mspec * 7.5 / 10)])
        else :
            lang_audio_features.extend(mspec)
        
    all_language_features[path.rsplit('/',1)[1]] = lang_audio_features

print np.shape(all_language_features['odia'])
print np.shape(all_language_features['kannada'])
print np.shape(all_language_features['telugu'])
print np.shape(all_language_features['bengali'])


(7795, 40)
(11391, 40)
(6591, 40)
(7831, 40)


In [101]:
from sklearn.mixture import GaussianMixture

all_language_gmm = {}

language_names = []

for key in all_language_features:
    all_language_gmm[key] = GaussianMixture(covariance_type='full' , n_components= 25).fit(all_language_features[key])
    language_names.append(key)
    print "fitting "+key

fitting odia
fitting bengali
fitting telugu
fitting kannada


In [93]:
print language_names

['odia', 'bengali', 'telugu', 'kannada']


In [104]:
from scipy import stats
import warnings

warnings.filterwarnings('ignore')

# testing...

path_to_test_audio_file = "/home/anjani/AMP/assignment_3/lang_rec_data_with_division/test"

total = {}
correct = {}
incorrect = {}

for lang in language_names:
    total[lang] = 0
    correct[lang] = 0
    incorrect[lang] = 0

for lang in language_names:
    
    audio_features = []
    all_audio_files = glob.glob(path_to_test_audio_file+"/"+lang+"/*.wav") 
    max_match = 0 
    actual_label = lang
    
    for i in all_audio_files:
        
        sample_rate, X = scipy.io.wavfile.read(i)
        ceps, mspec, spec = mfcc(X)
        num_mspec = len(mspec)
        if lang == 'odia' :
            audio_features.extend(mspec[int(3 * num_mspec / 10):int(num_mspec * 7.5 / 10)])
        else :
            audio_features.extend(mspec)
        lang_arry = []
        predict_lable = ""
        
        for one_row in audio_features :
            max_match = 0
            predicted_lable = ""
            for each_lang in all_language_gmm :
                score = np.exp(all_language_gmm[each_lang].score_samples(one_row))
                if score >= max_match:
                    max_match = score
                    predicted_lable = each_lang

            lang_arry.append(predicted_lable)
        predict_lable = max(set(lang_arry), key = lang_arry.count)
    
        if actual_label == predict_lable:
            correct[actual_label] = correct[actual_label]+1
            # print actual_label, predict_lable, "..... correct", correct[actual_label]
        else:
            incorrect[actual_label] = incorrect[actual_label] + 1
            # print actual_label, predict_lable, "..... incorrect", incorrect[actual_label]
            
        total[actual_label] += 1

print correct
print incorrect
print total

{'odia': 4, 'bengali': 2, 'telugu': 4, 'kannada': 3}
{'odia': 0, 'bengali': 2, 'telugu': 0, 'kannada': 1}
{'odia': 4, 'bengali': 4, 'telugu': 4, 'kannada': 4}


In [115]:
tot_correct = 0
tot_incorrect = 0

for lang in language_names:
    tot_correct += correct[lang]
    tot_incorrect += incorrect[lang]
accuracy = (1.0 * tot_correct)/(tot_correct + tot_incorrect)

print 'accuracy is : ',accuracy

accuracy is :  0.8125
